In [1]:
!pip install scikit-learn==1.2.2

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.base import clone
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train.drop(columns=['Listening_Time_minutes'])
y = np.log1p(train['Listening_Time_minutes'])
X_test = test[X.columns]

# Preprocessing
categorical = X.select_dtypes(include='object').columns.tolist()
numerical = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ]), numerical),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), categorical)
])

X_proc = np.asarray(preprocessor.fit_transform(X))
X_test_proc = np.asarray(preprocessor.transform(X_test))
y = np.asarray(y)

# Feature selection
selector = SelectKBest(score_func=f_regression, k=100)
X_proc = selector.fit_transform(X_proc, y)
X_test_proc = selector.transform(X_test_proc)


In [ ]:

# CNN K-Fold Training
def train_cnn_kfold(X, y, X_test, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    oof_preds = np.zeros(X.shape[0])
    test_preds = np.zeros(X_test.shape[0])
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
        X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        cnn = models.Sequential([
            layers.Conv1D(64, 5, activation='relu', input_shape=(X.shape[1], 1)),
            layers.BatchNormalization(),
            layers.Conv1D(64, 5, activation='relu'),
            layers.GlobalMaxPooling1D(),
            layers.Dropout(0.3),
            layers.Dense(64, activation='relu'),
            layers.Dense(1)
        ])

        cnn.compile(optimizer='adam', loss=tf.keras.losses.Huber())

        es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

        cnn.fit(X_train, y_train, validation_data=(X_val, y_val),
                epochs=50, batch_size=64, callbacks=[es], verbose=0)

        oof_preds[val_idx] = cnn.predict(X_val).flatten()
        test_preds += cnn.predict(X_test_cnn).flatten() / folds

    print(f"CNN CV RMSE: {np.sqrt(mean_squared_error(y, oof_preds)):.5f}")
    return oof_preds, test_preds

X_cnn = X_proc.reshape((X_proc.shape[0], X_proc.shape[1], 1))
cnn_oof, cnn_test_preds = train_cnn_kfold(X_proc, y, X_test_proc)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-15 18:41:06.350819: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-04-15 18:41:06.350876: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-04-15 18:41:06.350885: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-04-15 18:41:06.350903: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-15 18:41:06.350917: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] C

In [ ]:

# Model wrapper for tree models
class ModelWrapper:
    def __init__(self, model, name):
        self.model = model
        self.name = name
        self.oof = None
        self.models = []
        self.test_preds = None

    def fit_predict(self, X, y, X_test, folds=5):
        kf = KFold(n_splits=folds, shuffle=True, random_state=42)
        self.oof = np.zeros(X.shape[0])
        self.test_preds = np.zeros(X_test.shape[0])
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            model = clone(self.model)
            model.fit(X_train, y_train)
            self.oof[val_idx] = model.predict(X_val)
            self.test_preds += model.predict(X_test) / folds
            self.models.append(model)

        rmse = np.sqrt(mean_squared_error(y, self.oof))
        print(f"{self.name} CV RMSE: {rmse:.5f}")
        return self

# Tree models
lgb_model = ModelWrapper(
    lgb.LGBMRegressor(n_estimators=1200, learning_rate=0.01, num_leaves=64, max_depth=8,
                      subsample=0.8, colsample_bytree=0.8, random_state=42),
    "LightGBM"
)

xgb_model = ModelWrapper(
    xgb.XGBRegressor(n_estimators=1200, learning_rate=0.01, max_depth=6,
                     subsample=0.8, colsample_bytree=0.8, random_state=42),
    "XGBoost"
)

cat_model = ModelWrapper(
    cb.CatBoostRegressor(iterations=1200, learning_rate=0.01, depth=6,
                         verbose=0, random_seed=42),
    "CatBoost"
)

In [ ]:

# Train models
lgb_model.fit_predict(X_proc, y, X_test_proc)
xgb_model.fit_predict(X_proc, y, X_test_proc)
cat_model.fit_predict(X_proc, y, X_test_proc)

# Stacking
stacked_X = np.vstack([
    lgb_model.oof,
    xgb_model.oof,
    cat_model.oof,
    cnn_oof
]).T

stacked_test = np.vstack([
    lgb_model.test_preds,
    xgb_model.test_preds,
    cat_model.test_preds,
    cnn_test_preds
]).T

# Try different meta-models
meta_models = {
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.001),
    'SVR': SVR(),
    'GBR': GradientBoostingRegressor(n_estimators=200, learning_rate=0.05)
}

for name, meta_model in meta_models.items():
    meta_model.fit(stacked_X, y)
    preds = np.expm1(meta_model.predict(stacked_test))
    rmse = np.sqrt(mean_squared_error(np.expm1(y), preds[:len(y)]))  # Approx test error
    print(f"{name} MetaModel RMSE (approx): {rmse:.5f}")

# Or manual blend
final_preds = (
    0.3 * np.expm1(lgb_model.test_preds) +
    0.3 * np.expm1(xgb_model.test_preds) +
    0.3 * np.expm1(cat_model.test_preds) +
    0.1 * np.expm1(cnn_test_preds)
)

# Save submission
submission = pd.DataFrame({
    'id': test['id'],
    'Listening_Time_minutes': final_preds
})
submission.to_csv('ensemble_submission4.csv', index=False)
print("\n✅ Ensemble submission saved as 'ensemble4_submission.csv'")
